In [1]:
import numpy as np
from osgeo import gdal,ogr,osr
import geopandas as gpd
import glob,os,pathlib

In [2]:
image_path = "C:/Users/Ananthu/Desktop/Images/landsat/Landsat 8 44N"
cliped_path = "C:/Users/Ananthu/Desktop/Images/Testcode1"

In [23]:
List_of_images = glob.glob(image_path+'/*B*.TIF')

In [26]:
List_of_images.sort()

In [27]:
List_of_images

['C:/Users/Ananthu/Desktop/Images/landsat/Landsat 8 44N\\LC08_L2SP_143052_20220415_20220420_02_T1_ST_B10.TIF',
 'C:/Users/Ananthu/Desktop/Images/landsat/Landsat 8 44N\\LC08_L2SP_143052_20220415_20220420_02_T1_SR_B7.TIF',
 'C:/Users/Ananthu/Desktop/Images/landsat/Landsat 8 44N\\LC08_L2SP_143052_20220415_20220420_02_T1_SR_B6.TIF',
 'C:/Users/Ananthu/Desktop/Images/landsat/Landsat 8 44N\\LC08_L2SP_143052_20220415_20220420_02_T1_SR_B5.TIF',
 'C:/Users/Ananthu/Desktop/Images/landsat/Landsat 8 44N\\LC08_L2SP_143052_20220415_20220420_02_T1_SR_B4.TIF',
 'C:/Users/Ananthu/Desktop/Images/landsat/Landsat 8 44N\\LC08_L2SP_143052_20220415_20220420_02_T1_SR_B3.TIF',
 'C:/Users/Ananthu/Desktop/Images/landsat/Landsat 8 44N\\LC08_L2SP_143052_20220415_20220420_02_T1_SR_B2.TIF',
 'C:/Users/Ananthu/Desktop/Images/landsat/Landsat 8 44N\\LC08_L2SP_143052_20220415_20220420_02_T1_SR_B1.TIF']

In [5]:
def Create_boundBox(shape_save_path,projection):
    """
    shape_save_path : Path to which the created shape file is saved.
    projection : Cordinate reference of the shape obtained from the image.
    input for the cordinates are then given in input() fn according to the CRS
    
    """
    
    tllat = (float(input('Give TOP LEFT Lattitude in Decimal Degrees or northings in case of UTM:')))
    tllong = (float(input('Give TOP LEFT longitude in Decimal Degrees or eastings in case of UTM:')))

    brlat = (float(input('Give BOTTOM RIGHT Lattitude in Decimal Degrees or northings in case of UTM:')))
    brlong = (float(input('Give BOTTOM RIGHT Longitude in Decimal Degrees or eastings in case of UTM :')))

    Hdist = brlong-tllong
    Vdist = tllat-brlat
    shape_name = input('Give name for shape file(avoid invalied filenames):')
    #pointstllat,tllong
    #tllat,tllong,trlat,trlong,brlat,brlong,bllat,bllong =tllat,tllong ,tllat,tllong+Hdist ,brlat,brlong , tllat-Vdist,tllong
    print(tllat,tllong ,tllat,tllong+Hdist ,brlat,brlong , tllat-Vdist,tllong)


    ring = ogr.Geometry(ogr.wkbLinearRing) #(long,lat) anticlock
    ring.AddPoint(tllong,tllat-Vdist) #11.586101282798912, 77.63620736510678
    ring.AddPoint(brlong,brlat) #, 77.99657914761914
    ring.AddPoint(tllong+Hdist,tllat) #11.989962973654878, 78.00569199522926
    ring.AddPoint(tllong,tllat) #11.989962973654878
    ring.AddPoint(tllong,tllat-Vdist)

    poly = ogr.Geometry(ogr.wkbPolygon)
    poly.AddGeometry(ring)

    driver = ogr.GetDriverByName("ESRI Shapefile")

    outds = driver.CreateDataSource(f"{shape_save_path}/{shape_name}.shp") # give path to save .shp
    Outlayer = outds.CreateLayer("Study_Area",projection)
    feature = ogr.Feature(Outlayer.GetLayerDefn())
    feature.SetGeometry(poly)
    Outlayer.CreateFeature(feature)

    outds = Outlayer = feature = None

    path_to_shape = f"{shape_save_path}/{shape_name}.shp"
    
    return path_to_shape


In [22]:
def clip_with_cordinate(path_input,outPath,path_to_shape,shape_available=False):
    """
    path_input : Image to be clipped is given.
    outPath : Out put folder path. Only give path for directry to which you want to save images the function create folders
               and file  with names.
    shape_available : bool, wether shape file for clipping available or not. True if Yes False if no.
    path_to_shape : Path to the .shp file if it is available or path to save the created shape if it is not.
    
    """
    folderNm = pathlib.PurePath(path_input).name[:40]
    fileNm = pathlib.PurePath(path_input).name
    
    #print(path_to_shape)

    W_im = gdal.Open(path_input)
    rasterBand = W_im.GetRasterBand(1)
    elevArray = W_im.GetRasterBand(1).ReadAsArray().astype(np.float32)
    
    projection = osr.SpatialReference(wkt=W_im.GetProjection())
    #
    if shape_available == True:
        options = gdal.WarpOptions(cutlineDSName=path_to_shape,format="GTiff",cropToCutline=True,dstNodata=np.nan)
        outBand = gdal.Warp(srcDSOrSrcDSTab=path_input,
                            destNameOrDestDS=outPath+'/'+folderNm+'/Clipped_'+fileNm,
                            options=options)
        outBand= None
        
        print('saved at :',outPath+'/'+folderNm+'/Clipped_'+fileNm)
            
    else:
        shape_save_path = path_to_shape
        path_to_shape=Create_boundBox(shape_save_path,projection)
        
        
        options = gdal.WarpOptions(cutlineDSName=path_to_shape,format="GTiff",cropToCutline=True,dstNodata=np.nan)
        outBand = gdal.Warp(srcDSOrSrcDSTab=path_input,
                            destNameOrDestDS=outPath+'/'+folderNm+'/Clipped_'+fileNm,
                            options=options)
        
        print('saved at :',outPath+'/'+folderNm+'/Clipped_'+fileNm)
        shape_available = True
        
    return path_to_shape ,shape_available
        
        

        #path_to_shape = input('give path to shape:')
               

In [ ]:
def Clip_to_bound(In_path,shp_path,output_path):
    
    folderNm = pathlib.PurePath(path_input).name[:40]
    fileNm = pathlib.PurePath(path_input).name
    
    options = gdal.WarpOptions(cutlineDSName=shp_path,format="GTiff",cropToCutline=True,dstNodata=np.nan)
    outBand = gdal.Warp(srcDSOrSrcDSTab=In_path,
                        destNameOrDestDS=output_path+'/'+folderNm+'/Clipped_'+fileNm,
                        options=options)
    outBand= None
    
    print('saved at :',outPath+'/'+folderNm+'/Clipped_'+fileNm)
        

In [1]:
def Clip_to_bound1(In_path,shp_path,output_path):
    
    #folderNm = pathlib.PurePath(path_input).name[:40]
    #fileNm = pathlib.PurePath(path_input).name
    
    options = gdal.WarpOptions(cutlineDSName=shp_path,format="GTiff",cropToCutline=True,dstNodata=np.nan)
    outBand = gdal.Warp(srcDSOrSrcDSTab=In_path,
                        destNameOrDestDS=output_path,
                        options=options)
    outBand= None
    

In [16]:
file = List_of_images[0]
file

'C:/Users/Ananthu/Desktop/Images/landsat/Landsat 8 44N\\LC08_L2SP_143052_20220415_20220420_02_T1_SR_B1.TIF'

In [19]:
#folderNm = pathlib.PurePath(file).name[:40]
#fileNm = pathlib.PurePath(file).name

# Clip images in a folder
shape_available1=bool(int(input("Is shp file available?:type-> 1 for True, 0 for False : "))) 
path_to_shape1 = input("Give valied path to shape file or folder to save shape file : ")
for file in List_of_images:
    folderNm = pathlib.PurePath(file).name[:40]
    try:
        os.stat(cliped_path+'/'+folderNm)
    except:
        os.mkdir(cliped_path+'/'+folderNm)
    else:
        pass
    #print(path_shape1)
    #os.mkdir(out_path+'/'+folderNm)
    path,stat = clip_with_cordinate(file,cliped_path,path_to_shape1,shape_available1)
    
    path_to_shape1 = path
    shape_available1 = stat

Is shp file available?:type-> 1 for True, 0 for False : 0
Give valied path to shape file or folder to save shape file : C:/Users/Ananthu/Desktop/Images/Testcode1
Give TOP LEFT Lattitude in "Decimal Degrees":1326984
Give TOP LEFT longitude in "Decimal Degrees":133286
Give BOTTOM RIGHT Lattitude in "Decimal Degrees":1282929
Give BOTTOM RIGHT Longitude in "Decimal Degrees":173664
Give name for shape file(avoid invalied filenames):test8
1326984.0 133286.0 1326984.0 173664.0 1282929.0 173664.0 1282929.0 133286.0
saved at : C:/Users/Ananthu/Desktop/Images/Testcode1
saved at : C:/Users/Ananthu/Desktop/Images/Testcode1
saved at : C:/Users/Ananthu/Desktop/Images/Testcode1
saved at : C:/Users/Ananthu/Desktop/Images/Testcode1
saved at : C:/Users/Ananthu/Desktop/Images/Testcode1
saved at : C:/Users/Ananthu/Desktop/Images/Testcode1
saved at : C:/Users/Ananthu/Desktop/Images/Testcode1
saved at : C:/Users/Ananthu/Desktop/Images/Testcode1


In [56]:
shp_path = 'C:/Users/Ananthu/Desktop/Images/Testcode1/shpp/Bound.shp'

In [ ]:
Give TOP LEFT Lattitude in "Decimal Degrees":11.989962973654878
Give TOP LEFT longitude in "Decimal Degrees":77.63620736510678
Give BOTTOM RIGHT Lattitude in "Decimal Degrees":11.586101282798912
Give BOTTOM RIGHT Longitude in "Decimal Degrees":78.00569199522926
Give name for shape file(avoid invalied filenames):test

In [42]:
cliped_path+'/'+folderNm

'C:/Users/Ananthu/Desktop/Images/Testcode1/LC08_L1TP_144053_20210302_20210311_02_T1'

In [ ]:
#####LONG ### LAT
BR = 173664,1282929
TL = 133286,1326984
BL = 133286,1282929
TR = 173664,1326984

In [84]:

try:
    os.stat(image_path)
except:
    ans=False
else:
    ans=True

In [60]:
pathlib.PurePath(file).name

'LC08_L2SP_143052_20220415_20220420_02_T1_QA_PIXEL.TIF'

In [67]:
List_of_images

['C:/Users/Ananthu/Desktop/Images/landsat/Landsat 8 44N\\LC08_L2SP_143052_20220415_20220420_02_T1_SR_B1.TIF']

In [96]:
pathlib.PurePath(List_of_images[0]).name[:40]

'LC08_L1TP_144053_20210302_20210311_02_T1'

In [21]:
folderNm

'LC08_L2SP_143052_20220415_20220420_02_T1'

In [114]:
def conadd(a,b,addt = False):
    if addt==True:
        
        c =  a+b
    else:
        c = a+b+a
        
        addt = True
    return c,addt

In [117]:
c=1
addt=bool(input())
for i in range(3):
    print(i)
    print('c is',c)
    print(addt)
    
    c1,a =conadd(2,1,addt)
    c=c1
    addt=a
    


0
c is 1
False
1
c is 5
True
2
c is 3
True


In [11]:
a=''
a=a+'dsvsdv'

In [12]:
a

'dsvsdv'

In [28]:
shape_name = 'bijno'
shape_save_path=input()#'C:/Users/Ananthu/Desktop/Images/Testcode1'

'C:/Users/Ananthu/Desktop/Images/Testcode1'


In [29]:
f"{shape_save_path}/{shape_name}.shp"

"'C:/Users/Ananthu/Desktop/Images/Testcode1'/bijno.shp"

In [2]:
('LC08_L2SP_143052_20210530_20210608_02_T1')[:-4]

'LC08_L2SP_143052_20210530_20210608_0'